In [ ]:
%pip install -r requirements.txt
%pip install "deepspeed==0.14.0" --upgrade
%pip install modelscope
%pip install flash-attn==2.5.8 --no-build-isolation
# %DS_BUILD_CPU_ADAM=1  BUILD_UTILS=1  pip install deepspeed==0.14.3 -U
# %DS_BUILD_CPU_ADAM=1  BUILD_UTILS=1  pip install deepspeed==0.14.0 -U
# %conda install pytorch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 pytorch-cuda=12.1 -c pytorch -c nvidia

# %pip install jieba
# %pip install prettytable
# %pip install accelerate --upgrade
# %pip uninstall -y transformers
# %pip install git+https://github.com/huggingface/transformers


In [ ]:
dbutils.library.restartPython()

In [ ]:

import transformers
import torch
import subprocess

from modelscope import snapshot_download

base_path = "/Volumes/main/default/default_volume/erikyzzhang/long_context"
data_path = base_path + "/data"
model_path = base_path + "/models"

# model_id = snapshot_download("LLM-Research/Meta-Llama-3.1-8B-Instruct", cache_dir=model_path)# 移动模型文件到models目录

# subprocess.run(['mv', base_path + '/models/LLM-Research/Meta-Llama-3-8B-Instruct', base_path + '/models/llama3-8B-8k'], check=True)

subprocess.run(['modelscope', 'download', '--model=LLM-Research/Meta-Llama-3.1-8B-Instruct', '--local_dir', model_path + '/llama-3.1-8B-128k-Instruct'], check=True)
# huggingface-cli download meta-llama/Meta-Llama-3.1-8B --include "original/*" --local-dir Meta-Llama-3.1-8B

In [ ]:
import os
environment= "databricks" # databricks / terminal

生成数据集

In [ ]:

if environment == "databricks":
    base_path = "/Volumes/main/default/default_volume/erikyzzhang/long_context_decoding"
    source_file = base_path + "/motivation" + '/kv-retrieval-3000_keys.jsonl'
    save_dir = base_path + "/motivation/data"
    
elif environment == "terminal":
    source_file = './benchmark/super_retrieval/kv-retrieval-3000_keys.jsonl'
    save_dir = './motivation/data'


if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
!python motivation/generate_dataset.py --source_file {source_file} --save_dir {save_dir}

生成概率分布

In [ ]:
if environment == "databricks":
    base_path = "/Volumes/main/default/default_volume/erikyzzhang/long_context_decoding"
    model_path = '/Volumes/main/default/default_volume/erikyzzhang/long_context/models/llama-3-8B-262k'
    data_path = base_path + "/motivation/data"
    
elif environment == "terminal":
    model_path = '/home/zikaixiao/zikaixiao/LongLoRA-main/models/llama-3-8B-262k'
    data_path = './motivation/data'

!python motivation/generate_prob.py --model_path {model_path} --data_path {data_path}

计算Mean Reciprocal Rank

In [ ]:
if environment == "databricks":
    base_path = "/Volumes/main/default/default_volume/erikyzzhang/long_context_decoding"
    model_path = '/Volumes/main/default/default_volume/erikyzzhang/long_context/models/llama-3-8B-262k'
    data_path = base_path + "/motivation/data"
    
elif environment == "terminal":
    model_path = '/home/zikaixiao/zikaixiao/LongLoRA-main/models/llama-3-8B-262k'
    data_path = './motivation/data_32k'

!python motivation/generate_result.py --model_path {model_path} --data_path {data_path}